### Project Title: Automate Data Standarization of Ad-Spend Competitive Data
    
Project Category: Automation in Python
Project Objective/Purpose: Client would like to know the market share of ad-spend and sales for its brands and 
    its comeptitors to make media spend decision accross differnet markets.
Project Procedue: 
    Data Collection, Data pre-processing, Data Analysis
Project Outcome: Automation, Reduce Data pre-processing Time for Analysis

In [1]:
# Import Libraries:
import time
start_time = time.time()
import datetime
import os
import re
import openpyxl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
# filter warning codes
warnings.filterwarnings("ignore")

In [2]:
# Fucntion to get data files & merge into one dataframe
path = r"C:\ABI\2023 Competitive\2023 Competitive through Q4\Ad Spend\Raw Data 2"
def read_files_merge_data(path):
    
    """ 
        This function reads multiple excel files with same column headers from 
        a specific path from the system,and all the files are merged into a master dataframe.
     
     """
    
    path_ = os.chdir(path)
    files = os.listdir(path_)
    
    merge_data = pd.DataFrame() # Create an empty
    for f in files:
        data = pd.read_excel(f)
        merge_data = merge_data.append(data)
    return merge_data

# Execute Fuction:
df_initial = read_files_merge_data(path)
df_initial.shape

(507209, 11)

In [3]:
# Function for Column Name mapping:
def column_rename(dataframe):
    
    """ 
        This fucntion reads the dataframe, standarize the columns names as per the pre-defined requriement 
    i.e. using the variable  'col_name_mapping' mentioned within the function.
    
    """
    
    col_name_mapping = {'Country': 'Country', 'Data Source': 'Data_Source',
                   'Advertiser (parent company)': 'Advertiser_parent_company', 'Brand': 'Brand',
                   'Category':'Category', 'Media type': 'Media_type', 'Year': 'Year', 'Month': 'Month',
                   'Spend (Local Currency)':'Spend_Local_Currency', 'Exchange Rate': 'Exchange_Rate',
                   'Spend (US Currency)': 'Spend_US_Currency'}
    dataframe = dataframe.rename(columns = col_name_mapping)
    return dataframe

df = column_rename(df_initial)

df.shape

(507209, 11)

In [4]:
df.head()

,Country,Data_Source,Advertiser_parent_company,Brand,Category,Media_type,Year,Month,Spend_Local_Currency,Exchange_Rate,Spend_US_Currency
0,Argentina,IBOPE - AdMedia,500 CIDER,500 CIDER,SIDRAS,Vía Pública,2023,11,2.253914e+06,253.93,8876.125086
1,Argentina,IBOPE - AdMedia,7 COLINAS CERVEZA ARTESAN,7 COLINAS,CERVEZAS ARTESANALES,Internet,2023,2,8.996066e+03,253.93,35.427348
2,Argentina,IBOPE - AdMedia,7 COLINAS CERVEZA ARTESAN,7 COLINAS,CERVEZAS ARTESANALES,Internet,2023,4,2.260157e+04,253.93,89.007094
3,Argentina,IBOPE - AdMedia,7 COLINAS CERVEZA ARTESAN,7 COLINAS,CERVEZAS ARTESANALES,Internet,2023,5,2.223178e+04,253.93,87.550835
4,Argentina,IBOPE - AdMedia,7 COLINAS CERVEZA ARTESAN,7 COLINAS,CERVEZAS ARTESANALES,Internet,2023,6,5.869406e+04,253.93,231.142678


In [5]:
# Function - String Value Upper case
def string_upper(df): 
    
    """
    Function converts certain column values into upper string, and remove extra spaces if any 
    from the begining & end of the string.
    
    """
    
    for col in df.columns[1:-7]:
        df[col] = df[col].str.upper()
        df[col] = df[col].str.strip()
    return df
df = string_upper(df)

### Available Markets

In [6]:
# Check all the avaibale markets/country names
df['Country'].unique()

array(['Argentina', 'Belgium', 'Bolivia', 'Brazil', 'Canada', 'Chile',
       'China', 'Colombia', 'Dominican Republic', 'Ecuador',
       'El Salvador', 'France', 'Guatemala', 'Honduras', 'India', 'Japan',
       'Mexico', 'Netherlands', 'Panama', 'Paraguay', 'Peru',
       'South Africa', 'South Korea', 'UK', 'Uruguay', 'US', 'Vietnam'],
      dtype=object)

In [7]:
## Creating requried columns with null values
df["Category_Standardized"] = np.nan
df["Advertiser_parent_standarized"] = np.nan
df["Brand_standarization"] = np.nan
df['Parent_competitor'] = np.nan
df["Top_Brand_Competitors_and_ABI"] = np.nan
df["Focused_Brand_Competitors"] = np.nan
df['Top10_Brands_by_Market'] = np.nan
df["Media_Summary"] = np.nan
df["Media_type_standarized"] = np.nan
df["Month_number"] = np.nan
df['Month_standarization'] = np.nan
df["Focused_Global_Brand"] = np.nan
df["Global_Parent"] = np.nan
df["ABI_Parent_SOS"] = np.nan
df["Heineken_Brand_SOS"] = np.nan
df["Amstel_Brand_SOS"] = np.nan
df["Digital_Traditional"] = np.nan
df["Heineken_Parent_SOS"] = np.nan
df["Brand_Rank_by_Country"] = np.nan
df["Market_Group"] = np.nan

### Category (standardized)

In [11]:
# Drink Categorization:
 
"""
    Drink category featrue should converted in to four main categories, 
    i.e. "Beer", "Beyond Beer", "Non Alcholic Beer", "Other Alcohol" 
    Mapp the eith of the four new values in the new column called "Category_Standardized".
    """

beer = ["MALTAS", 'MALTA', 'MALT', "BEER", "BEER - OTHER", "CERVEZA", "CERVEZAS", "CERVEZAS ABI", 
        "CERVEZAS ARTESANALES", "CERVEZAS CCC", "CERVEZAS RESTANTES", "EVENTS DRINKS", "INST. BAVARIA", 'BEER RANGES',
       'Light & Low Alcohol Beer & Ale', 'Regular Beer & Ale', 'Cerveza', "Alcoholic Drink - Brand Building", 
       "Alcoholic Drink - Sports Sponsorship", "Alcoholic Drink - Sports Sponsorship", "Ales & Bitters", 
       "Beer Lager Cider - Multi Product", "Lagers", "Stouts", 'BIERES MULTI-PRODUITS', 'BIERES SPECIALES', 'PILS',
       'PILS DE LUXE', 'Lager', 'Alcoholic Drink-Brand Building', 'Ales&Bitters', 'Stout', 'Ready Mixed/alcopops ', 
       'BIRRE ALCOOLICHE', 'BIRRE ANALCOOLICHE', 'EVENTI E SPETTACOLI', 'CERVEZA ARTESANAL', 'CERVEZA LIGHT', 
        'BIERES BLANCHES', 'CONCERTS - MANIFESTATIONS MUSICALES', 'Drink - Brand Building', 'BIRRE',
       'Beer Lager & Cider', 'Beer Lager & Cider'.upper(), 'CHICHA', 'READY TO DRINK', 'LIGHT BEER', 'CERVEJAS',
        'OTHER BEERS', 'ABI BEERS', 'BAVARIA INSTITUTIONAL', 'CCC BEERS', 'Beer - Other', 'MEZCLAS RTD',
        'Beer & Ale: Comb Copy & NEC']


beyond_beer = ["SELTZER", 'beyond Beer', "COOLERS, READY TO DRINK", "Ready Mixed/alcopops"]

non_alcoholic_beer = ['Non Alc', "Alcohol Free Beer Lager Cider", 'Beer Non Alcoholic,- Arm', 
                      'INST BEBIDAS NO ALCOHOLICAS', 'INSTITUC BEBIDAS', 'Beer & Wine Corporate Promotion', 
                      'Beer & Wine NEC']

other_alcohol = ["Cider/perry", "ENERGY DRINK", "Energy drink", "COLAS", "E-COMMERCE", "FERNETS", "GIN", 
                 "INSTITUC BEBIDAS ALCOHOLICAS", 
                 "JUGOS NATURALES", "LIMON DIET", "LINEA SABORES", "MANZANA", "NARANJA", "SIDRAS"," VINOS ALTA GAMA", 
                 "VINOS BAJA GAMA", "VINOS ESPUMANTES", "VINOS MEDIA GAMA", "Liqueur", "LIQUEURS"," Non-Beer", 
                 "OTHER", "Other Alcoholic Drinks", "OTHER LIQUORS", "OTROS LICORES", "RON", "RTD ALCOHOLIC BEVERAGES",
                 "RTD MIXES", "RUM", "Soda", "SODAS", "SPIRITS", "SPIRITS - OTHER", "UVA", "VINO", "VINOS", 
                 "VINOS FRIZZANTE", "VODKA", "WHISKY", "WHISKYS", "WINE", "WINE - OTHER", "WINE/SPARKLING WINE",
                 "AGUAS MINERALES Y MINERALIZADA", "AGUAS SABORIZADAS CON GAS", "AGUAS SABORIZADAS LINEA",
                 "AGUAS SABORIZADAS SIN GAS", "ALCOHOLIC BEVERAGES", "CAMPAÑAS DE CONSUMO ALCOHOLIC", "CHAMPAGNE",
                 "Destilled Spirit", "INSTITUC BEBIDAS SIN ALCOHOL", "LICORES", "LIMA LIMON, CITRUS",
                 "LINEA BEBIDAS ALCOHOLICAS", "LINEA BEBIDAS SIN ALCOHOL", "APERITIV SIN ALCOHOL Y VEGETAL",
                 "BEBIDAS ALCOHOLICAS PREPARADAS", "BEBIDAS ALCOHOLICAS SABORIZADAS", "DISTRIBUID BEBIDAS ALCOHOLICAS",
                 "FLAVORED ALCOHOLIC BEVERAGES", "GRAPAS", "IND. LICORERA", "AGUARDIENTE", "ALCOPOPS", "APERITIVOS",
                 "BEBIDAS DE SOJA", "CIDER", "CIDER - OTHER", "COCTELES", "COLAS DIET", "EMBOTELLADORAS", "FRUTAS",
                 "JUGOS EN POLVO PARA PREPARAR", 'VINOS ALTA GAMA', 'NON-BEER', 'LIQUEUR', 'DESTILLED SPIRIT', 
                 'OTHER ALCOHOLIC DRINKS', 'SODA', 'Cider', 'Wine', 'Alcoholic Beverages: Comb Copy & NEC', 'Non-Beer', 
                 'AGUA ARDIENTE', 'COGNACS', 'RONES', 'TIENDAS DE LICORES', 'VODKAS', 'WHISKEY', 
                 'COCKTAILES/MEZCLADORES', 'AUTRES MANIFESTATIONS', 'MANIFESTATIONS SPORTIVES', 
                 'SPIRITUEUX MULTI-PRODUITS', 'ROMPOPE',  'AGUARDIENTES', ' ', 'TEQUILAS', 'VINOS Y ESPUMANTES', 
                'INST BEBIDAS ALCOHOLICAS', 'COCKTAILS Y ESPUMANTES', 'DISTRIBUID BEBIDAS SIN ALCOHOL',
                'POMELO', 'TONICAS', np.nan, 'FIZZ', 'Spirits & Liqueurs', 'Other Alcohol', 'LIQUOR', 'SINGANI',
                'OTHER DRINKS WITH ALCOHOL', 'Sodas', 'LINHA BEBIDAS', 'HARD LIQUOR',
                'WINES', 'LIQUOR INDUSTRY', 'BEBIDAS ALCOHÓLICAS', 'LIQUERS']

df["Category_Standardized"] = df['Category'].copy()
df["Category_Standardized"] = df["Category_Standardized"].replace(beer, "Beer")
df["Category_Standardized"] = df["Category_Standardized"].replace(beyond_beer, "Beyond Beer")
df["Category_Standardized"] = df["Category_Standardized"].replace(other_alcohol, "Other Alcohol")
df["Category_Standardized"] = df["Category_Standardized"].replace(non_alcoholic_beer, "Non Alcoholic Beer")
df['Category_Standardized'].unique()

array(['Other Alcohol', 'Beer', 'Beyond Beer',
       'EVENTS & SPORT SPONSORSHIP', 'HARD SELTZER', 'Non Alcoholic Beer',
       'Ind. Licorera', 'Vinos', 'Ron', 'Other Beers', 'Whisky',
       'Beers Ccc', 'Aguardiente', 'Vodka', 'Other Liquors', 'Rtds',
       'Ready Mixed/Alcopops', 'Alcohol Drink Brand Building',
       'Alcohol Free Beer Lager  Cider', 'lager', 'Spirits', 'NAB',
       'Retailer', 'RTD+Seltzers'], dtype=object)

### Advertiser (standardized)

In [12]:
## Dictionary - Advertizer parent company
"""
    Differnet Beer manufacturing companies (Parent Companies) are registered
    in differnt names across differnt market/region. Thus all of this similarities 
    should be mapped under unique global parent company. i.e. All associated 
    "ANHEUSER-BUSCH INBEV" company in any market should be standarized as "ABI"
"""

df["Advertiser_parent_standarized"] = df["Advertiser_parent_company"].copy()

df.loc[(df['Country'] == 'Colombia') & (df['Advertiser_parent_standarized'] == "BAVARIA"), 
       'Advertiser_parent_standarized'] = "ABI"

advertiser_dictionary = { "QUILMES": "ABI", "ANHEUSER-BUSCH INBEV" : "ABI", "GOOSE ISLAND BEER COMPANY" : "ABI",
                   "ABINVEB" : "ABI", "CERVECERIA CHILE" : "ABI", "PATAGONIA (CERVEZA)" : "ABI",
                   "CLUB COLOMBIA CABLE PANREGIONAL" : "ABI", "QUILMES CABLE PANREGIONALL" : "ABI",
                   "BUDWEISER CABLE PANREGIONAL" : "ABI", "MIKES HARD CABLE PANREGIONAL" : "ABI",
                    "Budweiser Brewing" : "ABI", "Budweiser Brewing".upper() : "ABI", 
                        "ABINBEV" : "ABI", 'AMBEV' : "ABI", 
                         'ANHEUSER-BUSCH' : "ABI",
                         
                   "AGUILA CABLE PANREGIONAL" : "ABI", "CIA. DE CERVEZAS NACIONALES" : "ABI",
                   "STELLA ARTOIS" : "ABI", "INDUSTRIAS LA CONSTANCIA SA DE CV" : "ABI",  
                   "AMBEV CENTROAMERICA" : "ABI", "ANHEUSER-BUSCH CABLE PANREGIONAL" : "ABI", 
                   "GRUPO MODELO CABLE PANREGIONAL" : "ABI", "MIKES HARD LEMONADE CABLE PANREGIONAL" : "ABI", 
                   "CERVECERIA HONDUREÑA" : "ABI", "CROWN BEERS INDIA LTD" : "ABI", "DIST COMERCIAL GROUP" : "ABI", 
                   "UCP BACKUS Y JOHNSTON S.A.A." : "ABI", "CERVECERIA MODELO" : "ABI", "BELGIUM BEER COMPANY" : "ABI",
                   "ANHEUSER BUSCH INC" : "ABI", "CORONA CHASING SUNSETS" : "ABI", "ABInBev" : "ABI", "FNC" : "ABI",
                   "FABRICAS NACIONALES DE CERVEZA" : "ABI", "CORONA" : "ABI", "ANHEUSER-BUSCH INBEV" : "ABI",
                   "SAB MILLER" : "ABI", "Anheuser-Busch InBev" : "ABI", "CERVECERIA BOLIVIANA NACIONAL S.A." : "ABI",
                   "AB INBEV UK LTD" : "ABI", "ANHEUSER BUSCH INBEV" : "ABI", "AB INBEV" : "ABI", 
                   "ANHEUSER-BUSCH INBEV GERM.HOL., BREMEN" : "ABI", "AB INBEV IT.SPA GALLARATE VA" : "ABI",
                   "AB INBEV NEDERLAND BREDA" : "ABI", 
                   "CAMDEN TOWN BREWERY LTD" : "ABI", 'AMSTEL' : "ABI", 'CERVECERIA NACIONAL DOMINICANA' : "ABI",
                         "CERVEPAR" : "ABI",
                        # "BROUWERIJ BOSTEELS BUGGENHOUT [BE]" : "ABI",
                         
                    "HEINEKEN INT" : "HEINEKEN", "UNITED BRANDS" : "HEINEKEN", "HEINEKEN CABLE PANREGIONAL" : "HEINEKEN",
                    "Heineken" : "HEINEKEN", "DISAL" : "HEINEKEN", 
                    "CERVECERIA CENTRO AMERICANA" : "HEINEKEN", "DISTRIBUIDORA ISTMANIA" : "HEINEKEN", 
                         "HEINEKEN N.V." : "HEINEKEN", "HEINEKEN HOLDING NV" : "HEINEKEN", 
                         "Bebidas del Paraguay" : "HEINEKEN", "CERVECERIA PANAMA" : "HEINEKEN",
                         "LIMBA SERVICES SAC" : "HEINEKEN", "HEINEKEN INTERNATIONAL" : "HEINEKEN", 
                         "HEINEKEN SOUTH AFRICA" : "HEINEKEN", "HEINEKEN NV" : "HEINEKEN",  "HEINEKEN UK LTD" : "HEINEKEN",
                         "HEINEKEN COMPANY" : "HEINEKEN", "HEINEKEN FRANCE GPE" : "HEINEKEN", 
                         "HEINEKEN DT., BER" : "HEINEKEN", "HEINEKEN ITALIA SPA MILANO" : "HEINEKEN", 
                         "Heineken Nederland (WHK) Zoeterwoude".upper() : "HEINEKEN", 
                         "COMPANIA CERVECERIAS UNIDAS" : "HEINEKEN", 
                         "HEINEKEN VIETNAM BREWERY LIMITED COMPANY" : "HEINEKEN",
                         'HEINEKEN ENTREPRISE' : "HEINEKEN", "Heineken Ireland" : "HEINEKEN", 
                         'HEINEKEN IRELAND' : "HEINEKEN", "DISTRIBUIDORA DEL PARAGUAY" : "HEINEKEN",
                         'HEINEKEN HOLDING N.V' : "HEINEKEN", 'HEINEKEN N.V' : "HEINEKEN",
                         
                        "CCU SA" : "HEINEKEN", "CR BEER" : "HEINEKEN", 
                         "CCU CABLE PANREGIONAL" :  "HEINEKEN",
                         "CCC" : "HEINEKEN", "VBL" : "HEINEKEN",
                         
                         "CARLSBERG BEER" : "CARLSBERG", "CARLSBERG FOUNDATION" : "CARLSBERG",
                         "CARLSBERG VIETNAM" : "CARLSBERG", "CARLSBERG BREWERIES A/S": "CARLSBERG",
                         "CARLSBERG UK LTD" : "CARLSBERG", 
                         "CARLSBERG IMPORTERS" : "CARLSBERG", "CARLSBERG GROUPE" : "CARLSBERG",
                         "CARLSBERG IT.INDUNO O.VA" : "CARLSBERG", 
                         "CARLSBERG DRINKS RANGE" : "CARLSBERG", 'CARLSBERG GROUP' : "CARLSBERG",
                         #"CARLSBERG DT., HH" : "CARLSBERG",
                         
                         "BAVARIA BRASSERIE" : "BAVARIA", "BAVARIA BRAU PTY LTD" : "BAVARIA", 
                         "BAVARIA CABLE PANREGIONAL" : "BAVARIA", "BAVARIA ITALIA SRL TORINO" : "BAVARIA", 
                          "BAVARIA BRASSERIE" : "BAVARIA", "BAVARIA BREWERY" : "BAVARIA", 
                         "BAVARIA ITALIA SRL TORINO" : "BAVARIA",
                        # "BAVARIA LIESHOUT" : "BAVARIA",
                         
                         "bacardi martini india" : "BACARDI", "DESTILERIA PERUANA" : "BACARDI", 
                         "BACARDI CABLE PANREGIONAL" : "BACARDI", "BACARDI LIMITED" : "BACARDI", 
                         "BACARDI LTD" : "BACARDI", 
                         
                         "ASAHI BREWERIES LTD" : "Asahi", "ASAHI UK LTD" : "ASAHI", "ASAHI UK" : "ASAHI",
                          "Asahi Uk" : "ASAHI",
                         "ASAHI - SUPER DRY BEER" : "ASAHI", 
                         #"ASAHI BRANDS GERMANY, KOELN" : "ASAHI",
                         
                          "MOLSON COORS CHILE" : "MOLSON COORS", 'MOLSON COORS BREWING CO' : "MOLSON COORS",
                         'MOLSON' : "MOLSON COORS", "MOLSON COORS BREWING CO UK LTD" : "MOLSON COORS",
                         "Distribuidora Gloria" : "MOLSON COORS", "GOLDEN BLUE" : "GOLDEN BLUE (MOLSON COORS)",
                         'COORS BREWING COMPANY' : "MOLSON COORS", 'MILLER' : "MOLSON COORS",
                         'MOLSON COORS' : "MOLSON COORS", 'COORS BREWING' : "MOLSON COORS",
                         "Molson Coors Europe" : "MOLSON COORS", "Molson Coors Europe".upper() : "MOLSON COORS", 
                         
                         "Pernod Ricard india private limited" : "PERNOD RICARD",
                         
                         "united spirits limited" : "United Spirits", 'UNITED BREWERIES GROUP' : "UNITED BREWERIES", 
                         'UNITED BREWERIES' : "UNITED BREWERIES",  'UNITED BREWERIES LIMITED' : "UNITED BREWERIES", 
                         
                         "HITE JINRO BEVERAGE" : "HITE JINRO",
                         
                         "DIAGEO UK LTD" : "DIAGEO", 'DIAGEO PLC' :  "DIAGEO", "Diageo" : "DIAGEO",
                         "BROWN FORMAN BEVERAGES WORLDWIDE/COCA COLA COMPANY" : "BROWN FORMAN",
                         "CONSTELLATION BRANDS INC" : "CONSTELLATION", 
                         "MARTIN'S GROUP" : "MARTINS", "DIAGEO COLOMBIA S.A." : "DIAGEO",
                         "DUVEL MOORTGAT BROUWERIJ" : "DUVEL", 
                         "BLUE RIBBON INTERMEDIATE HOLDINGS LLC" : "BLUE RIBBON", 
                         "BOSTON BEER CO" : "BOSTON BEER",
                         #"Duvel Moortgat Nederland Amsterdam".upper() : "DUVEL",
                         
                        # "WARSTEINER BRAUEREI HS.CRAMER, WARSTEIN" : "WARSTEINER",
                        # 'WARSTEINER NEDERLAND NIJMEGEN' : "WARSTEINER",
                        # "KROMBACHER BRAUEREI, KREUZTAL-KROMBACH" : "KROMBACHER",
                        # "PAULANER BRAUEREI, MUC" : "PAULANER",
                        # "VELTINS C.+.A. BRAUEREI, MESCHEDE" : "VELTINS",
                         #"FRIESISCHES BRAUHAUS ZU JEVER, JEVER" : "FRIESISCHES",
                       #  "RADEBERGER EXPORTBIERBRAUEREI, RADEBERG" : "RADEBERGER",
                         
                         #"Koninklijke Grolsch Enschede".upper() : "GROLSCH",
                        # "TEXELSE BIERBROUWERIJ OUDESCHILD" : "TEXELSE",
                         #"LIDL NEDERLAND HUIZEN" : "HUIZEN",                         
                         "OTHERS" : "All Others" }


df["Advertiser_parent_standarized"] = df["Advertiser_parent_standarized"].replace(advertiser_dictionary)

### Brand (standardized) 

In [13]:

"""
    Differnet Beer barnds under same parent companies in differnt market/region, should be
        standarized into same global brand name.
"""


df["Brand_standarization"] = df["Brand"].copy()

brand_dictionary = {"HEINEKEN 0 0" : "HEINEKEN 0.0", "HEINEKEN O.O LAGER" : "HEINEKEN 0.0", 
                    "HEINEKEN 0 0 A1PADEL PAYSPACE CAPE TOWN MASTER" : "HEINEKEN 0.0",
                    "HEINEKEN O.O LAGER" : "HEINEKEN 0.0", "HEINEKEN 00" : "HEINEKEN 0.0", 
                    'HEINEKEN BIER 0,0 PROZENT' : "HEINEKEN 0.0", 'HEINEKEN 0.0 ALCOHOLVRIJ BIER' : "HEINEKEN 0.0",
                    'HEINEKEN 0.0 BIRRA ANALC.' : "HEINEKEN 0.0", "HEINEKEN ZERO ALCOHOL" : "HEINEKEN 0.0",
                    "HEINEKEN MEXICO" : "HEINEKEN", "HEINEKEN LAGER" : "HEINEKEN", "HEINEKEN ()" : "HEINEKEN",
                    'UNITED BRANDS' : "HEINEKEN", "HEINEKEN - EURO 2020" : "HEINEKEN", 
                    "HEINEKEN - CHAMPIONS LEAGUE" : "HEINEKEN", "HEINEKEN - UEFA CHAMPIONS LEAGUE" : "HEINEKEN",
                    "HEINEKEN - HEINEKEN SILVER" : "HEINEKEN SILVER", "Heineken - Heineken Silver" : "HEINEKEN SILVER",
                    "HEINEKEN - UEFA EUROPA LEAGUE SPONSOR" : "HEINEKEN",
                    "HEINEKEN/TURKISH AIRLINES - UEFA CHAMPIONS LEAGUE" : "HEINEKEN", 
                    "HEINEKEN - BRAND BUILDING" : "HEINEKEN", "Heineken - Brand Building" : "HEINEKEN",
                    "HEINEKEN - UEFA CHAMPIONS LEAGUE SPONSOR" : "HEINEKEN", "Heineken - Champions League" :"HEINEKEN",
                    "Heineken - Champions League".upper() : "HEINEKEN",
                    "Heineken - Uefa Champions League Sponsor" : "HEINEKEN",
                    "Heineken - Uefa Champions League Sponsor".upper() : "HEINEKEN",
                    "HEINEKEN/OPPO - UEFA CHAMPIONS LEAGUE" : "HEINEKEN", 
                    "HEINEKEN - UEFA WOMENS CHAMPIONS LEAGUE SPONSOR" : "HEINEKEN",  
                    "Heineken - Euro 2020 Sponsorship" : "HEINEKEN", 
                    "Heineken - Euro 2020 Sponsorship".upper() : "HEINEKEN", 
                    "Heineken - Uefa Europa League Sponsor" : "HEINEKEN", 
                    "Heineken - Uefa Europa League Sponsor".upper() : "HEINEKEN", "Heineken - Uefa Sponsor" : "HEINEKEN",
                    "Heineken - Uefa Sponsor".upper() : "HEINEKEN", 
                    "Heineken - Uefa Womens Champions League Sponsor" : "HEINEKEN",
                    "Heineken - Uefa Womens Champions League Sponsor".upper() : "HEINEKEN",
                    "Heineken/expedia - Uefa Champions League" : "HEINEKEN",
                    "Heineken/expedia - Uefa Champions League".upper() : "HEINEKEN",
                    "Heineken/playstation - Uefa Champions League" : "HEINEKEN",
                    "Heineken/playstation - Uefa Champions League".upper() : "HEINEKEN",
                    'HEINEKEN LAGER BEER' : "HEINEKEN", 'HEINEKEN LAGER BEER RUGBY' : "HEINEKEN", 
                    'HEINEKEN BIER' : "HEINEKEN", 'HEINEKEN BIER RANGE' : "HEINEKEN", 
                    'HEINEKEN BIRRA' : "HEINEKEN", 'HEINEKEN LIN.BIRRA' : "HEINEKEN", 
                    'HEINEKEN SILVER BIRRA' : "HEINEKEN SILVER", 'HEINEKEN (ADV.) PRODUCTEN ALGEMEEN' : "HEINEKEN",
                    'HEINEKEN BIER PILS' : "HEINEKEN", 'HEINEKEN PROMOTIE' : "HEINEKEN", 
                    'HEINEKEN SILVER BIER PILS' : "HEINEKEN", 'HEINEKEN BIERE' : "HEINEKEN",
                    'HEINEKEN SILVER BIERE' : "HEINEKEN SILVER", 'CERVEZA HEINEKEN' : "HEINEKEN", 
                    'HEINEKEN (REG)' : "HEINEKEN", 'Heineken' : "HEINEKEN",  'Heineken Silver' : "HEINEKEN SILVER",
                     'HEINEKEN BEER' : "HEINEKEN", 'HEINEKEN ISLANDS EDGE IRISH STOUT' : "HEINEKEN",
                    "HEINEKEN BIRRA MORETTI" : "HEINEKEN", "HEINEKEN DRINKS RANGE" : "HEINEKEN",
                    "Heineken - 00 Alcohol Free Premium Lager Beer" : "HEINEKEN 0.0",
                    "Heineken - 00 Alcohol Free Premium Lager Beer".upper() : "HEINEKEN 0.0", 
                    "HEINEKEN EXTRA COLD" : "HEINEKEN", "HEINEKEN IMAGE/+PARTNER" : "HEINEKEN", 
                    "HEINEKEN IMAGE" : "HEINEKEN",
                    "HEINEKEN VERANSTALTUNGEN" : "HEINEKEN", 'HEINEKEN CERVEZA' : "HEINEKEN",

                    
                    'SAIGON BEER' : "SAIGON", 'Molson Coors' : "MOLSON COORS",  "MOLSON" : "MOLSON COORS", 
                    'Constellation' : "CONSTELLATION", "MOLSON COORS BEVERAGE CO" : "MOLSON COORS", 
                    'COORS LIGHT CERVEZA' : "COORS LIGHT",
                    
                    'AMSTEL BIERE COLLECTIVITE' : "AMSTEL",  'AMSTEL 0.0 ALCOHOLVRIJ BIER' : "AMSTEL 0.0",
                    'AMSTEL BIER PILS' :  "AMSTEL", 'AMSTEL RADLER' : "AMSTEL", 
                    'AMSTEL RADLER 0.0% ALCOHOLVRIJ BIER' : "AMSTEL 0.0", "GC/AMSTEL MALTA" : "AMSTEL MALTA",
                    
                    "CORONA." : "CORONA", 'CORONA (CERVEZA)' : "CORONA", 'CORONA CHASING SUNSETS' : "CORONA", 
                    'CERVEZA CORONA' : "CORONA", 'CORONA (REG)' : "CORONA",
                    'CORONA EXTRA BIERE' : "CORONA", "CORONA EXTRA BIER" : "CORONA",
                    'CORONA SUNSET BIERE' : "CORONA", 'CORONA EXTRA' : "CORONA", 
                    'CORONA SUNSET BIERE' : "CORONA", "CORONITA" : "CORONA", "CORONA EXTRA - LAGER" : "CORONA",
                    "CORONA - BRAND BUILDING" : "CORONA",  "CORONA - TROPICAL" : "CORONA", 
                    "CORONA TROPICAL" : "CORONA", 
                    "CORONA - CERO" : "CORONA", 'CORONA SUNSET FESTIVAL' : "CORONA", 
                    'CORONA OPEN JBAY' : "CORONA", 'CORONA BIRRA' : "CORONA", 'CORONA BIER RANGE' : "CORONA",
                    'CORONA BIER RANGE              VB' : "CORONA", 'CORONA EXTRA BIRRA' : "CORONA", 
                    'CORONA SUNSETS FESTIVAL EVENTI' : "CORONA", 'CORONA (CERVEZA)' : "CORONA", 
                    "CORONA EXTRA BEER" : "CORONA", "Corona Extra" : "CORONA", "CORONA CERO" :  "CORONA", 
                    "CORONA CERO 0,0 PROZ.ALKOHOLFREIES BIER" : "CORONA",  
                    "CORONA HARD SELTZER" : "CORONA",  
                    
                    "ASAHI UK LTD" : "ASAHI", 'ASAHI - SUPER DRY' : "ASAHI", "ASAHI - MANCHESTER FC SPONSOR" : "ASAHI",
                         "ASAHI - BEER" : "ASAHI", "ASAHI BEER" : "ASAHI",
                    
                    'BUDWEISER BEER' : "BUDWEISER", 'Budweiser Corporate'.upper() : "BUDWEISER", 
                    'Budweiser Sponsorship'.upper() : "BUDWEISER", 'BUDWEISER ()' : "BUDWEISER", 
                    "BUDWEISER BEER - BRAND BUILDING" : "BUDWEISER", 'BUDWEISER (REG)' : "BUDWEISER",
                    'BUD BIER PILS' : "BUDWEISER", 'BUDWEISER BUDVAR BIER PILS' : "BUDWEISER", 
                    "BUDWEISER BUDVAR BIER" : "BUDWEISER", "BUDWEISER BUDVAR ORIGINAL CZECH LAGER" : "BUDWEISER",
                    "BUDWEISER BUDVAR NEALKO ALKOHOLFREI" : "BUDWEISER",
                    'BUD BIERE' : "BUDWEISER", "BUD" : "BUDWEISER", "BUDWEISER LAGER" : "BUDWEISER",
                    "BUDWEISER - FIFA WORLD CUP SPONSOR" : "BUDWEISER", "Budweiser Beer" : "BUDWEISER",
                    "Budweiser Lager" : "BUDWEISER", "BUUUUUUD" : "BUDWEISER", 
                    "BUDWEISER BUDVAR - CZECH LAGER" : "BUDWEISER", "BUDWEISER BEER - LAGER" : "BUDWEISER",
                    "BUDWEISER BEER - WALKSAFE" : "BUDWEISER",
                   
                    "CARLSBERG - 00 ALCOHOL FREE LAGER" : "CARLSBERG", "CARLSBERG ALCOHOL FREE LAGER" : "CARLSBERG",
                    "CARLSBERG - LIVERPOOL FC SPONSOR" : "CARLSBERG", "CARLSBERG - DANISH PILSNER" : "CARLSBERG",
                    'CARLSBERG 0,0 ALKOHOLFREI' :  "CARLSBERG 0.0", 'CARLSBERG BEER IMAGE' : "CARLSBERG", 
                    'CARLSBERG BEER PILS' :  "CARLSBERG", 'CARLSBERG BRAUEREI BIER RANGE' :  "CARLSBERG", 
                    'CARLSBERG PREMIUM LAGER BIER' :  "CARLSBERG", 'CARLSBERG (T)' :  "CARLSBERG", 
                    'CARLSBERG BIRRA' :  "CARLSBERG", 'CARLSBERG LAGER' : "CARLSBERG", "CARLSBERG - LAGER" : "CARLSBERG",
                    
                    'CRISTAL ()' : "CRISTAL", 'CRISTAL (CERVEZA)' : "CRISTAL", 'CERVEZA CRISTAL' : "CRISTAL",
                    
                    'Dos Equis Corporate'.upper() : "DOS EQUIS", 'XX' : "DOS EQUIS", 
                    'DOS EQUIS XX BIERE' : "DOS EQUIS", 
                    
                    'STELLA ARTOIS BEER' : "STELLA ARTOIS", 'STELLA ARTOIS PLAYERS CHAMPIONSHIP' : "STELLA ARTOIS",
                    "STELLA ARTOIS - UNFILTERED" : "STELLA ARTOIS",
                    
                    'PATAGONIA CERVEZA' : "PATAGONIA", 'PATAGONIA (CERVEZA)' : "PATAGONIA", 
                    
                    'MILLER (CERVEZA)' : "MILLER", 'CERVEZA MILLER' : "MILLER", 
                    'Miller Lite Corporate'.upper() : "MILLER LITE", 'MILLER CERVEZA' : "MILLER", 
                    
                    "BIRRA MORETTI - ZERO BEER" : "MORETTI 0.0", "BIRRA MORETTI 0.0" : "MORETTI 0.0",
                    "BIRRA MORETTI - BRAND BUILDING" : "MORETTI", 'MORETTI BIRRA' : "MORETTI",
                    'MORETTI BIRRA WWW' : "MORETTI", 'MORETTI FILTRATA FREDDO BIRRA' : "MORETTI",
                    'MORETTI LIN.BIRRA' : "MORETTI", 'BIRRA MORETTI BIER PILS' : "MORETTI", 
                    'BIRRA MORETTI SALE DI MARE SPECIAALBIER' : "MORETTI", 
                    'BIRRA MORETTI - LAUTENTICA' : "MORETTI", 'BIRRA MORETTI' : "MORETTI", 
                    'BIRRA MORETTI LAUTENTICA LAGER' : "MORETTI", "BIRRA MORETTI - LAGER RANGE" :  "MORETTI",
                    
                    "LEFFE - BLONDE BEER" : "LEFFE", 'ABBAYE DE LEFFE BIERE' : "LEFFE", 
                    'LEFFE BIERES' : "LEFFE", "LEFFE CO.BIRRA" : "LEFFE", "LEFFE BIRRA" : "LEFFE",
                    "LEFFE CO.BIRRA" : "LEFFE",
                    
                    "BECKS - AUTONOMOUS" : "BECKS", "BECK'S BIRRA" : "BECKS", 
                    "BECK'S UNFILTERED BIRRA" :  "BECKS",
                    
                    "BRUGAL ()" : "BRUGAL", 
                    
                    'PACEÑA' : "PACENA", 
                    
                    'AUSTRAL (CERVEZA)' : "AUSTRAL", 'AUSTRAL (CERVEZA)': "AUSTRAL", 
                    
                    'SOL (CERVEZA)' : "SOL",
                    
                    'CUSQUEÑA' : "CUSQUENA", "COSTEÑA" : "COSTENA" , "Snow Others" : "SNOW", 
                    "Snow Others".upper() : "SNOW", 
                    
                    'CASTLE LITE BEER' : "CASTLE LITE", 'BRUTAL FRUIT SPRITZER' : "BRUTAL FRUIT",
                    
                    'CARLING BLACK LABEL BEER' : "CARLING BLACK LABEL", 
                    'CARLING BLACK LABEL BEER SOCCER' : "CARLING BLACK LABEL", 
                    'CARLING BLACK LABEL BEER SOCCER SPONSORSHIP' : "CARLING BLACK LABEL",
                    
                    'FLYING FISH CHILL BEER' : "FLYING FISH", 
                    'FLYING FISH PREMIUM FLAVOURED BEER' : "FLYING FISH",
                    'FLYING FISH FLAVOUR CHILLAS' : "FLYING FISH FLAVOUR CHILLAS", 
                    'FLYING FISH PREMIUM FLAVOURED BEER COMP' : "FLYING FISH",
                    'FLYING FISH SELTZER PROMOTION' : "FLYING FISH SELTZER",
                    
                    'HUNTERS CIDER COMPETITION' : "HUNTERS", 'HUNTERS CIDER RANGE' : "HUNTERS", 
                    'HUNTERS DRY CIDER' : "HUNTERS", 'HUNTERS REFRESH THE RULES OF IGRAND' : "HUNTERS",
                    
                    'SAVANNA CIDER MUSIC' : "SAVANNA", 'SAVANNA CIDER RANGE' : "SAVANNA", 
                    'SAVANNA COMICS AWARDS' : "SAVANNA", 'SAVANNA DRY CIDER' : "SAVANNA",
                    'SAVANNA DRY CIDER COMEDY' : "SAVANNA", 
                    
                    'THREE SHIPS WHISKY' : "THREE SHIPS", 'THREE SHIPS WHISKY COMPETITION' : "THREE SHIPS",
                    'THREE SHIPS WHISKY PROMOTION' : "THREE SHIPS", 
                  
                    'JOHNNIE WALKER BLACK LABEL' : "JOHNNIE WALKER", 
                    'JOHNNIE WALKER FLAVOURCODE' : "JOHNNIE WALKER", 
                    'JOHNNIE WALKER GOLD LABEL RESERVE' : "JOHNNIE WALKER",
                    'JOHNNIE WALKER WHISKY RANGE' : "JOHNNIE WALKER", 
                    
                    'BLACK CROWN PREMIUM GIN & TONIC' : "BLACK CROWN", 
                    
                    'Bud Light Corporate'.upper() : "BUD LIGHT", 'Bud Light Seltzer Corporate'.upper() : "BUD LIGHT",
                    'Bud Light Sponsorship'.upper() : "BUD LIGHT", "BUD LIGHT NEXT" : "BUD LIGHT",
                    
                    'Michelob Ultra Corporate'.upper() : "MICHELOB ULTRA", 
                    'Modelo Corporate'.upper() : "MODELO", 
                    
                    'MODELO ESPECIAL' : "MODELO", 'MODELO ORO' : "MODELO", 'MODELO NEGRA' : "MODELO", 
                    'MODELO NOCHE ESPECIAL' : "MODELO", 'MODELO LIGHT' : "MODELO",
                    'MODELO ESPECIAL CHELADA' : "MODELO", 'GRUPO MODELO' : "MODELO",  
                    'CERVEZA MODELO' : "MODELO",
                    
                    'CERVECERIA NACIONAL DOMINICANA' : "CERVECERIA",  
                    'CASTLE MILK STOUT MEDLEY PROMOTION' : "CASTLE MILK STOUT",
                    
                    'ABINBEV INSTITUCIONAL' : "ANHEUSER-BUSCH", 'CIA. CERVEZAS NACIONALES' : "ANHEUSER-BUSCH", 
                    'Anheuser-Busch'.upper() : "ANHEUSER-BUSCH", 
                    'Anheuser-Busch Corporate'.upper() : "ANHEUSER-BUSCH", 'AB INBEV' : "ANHEUSER-BUSCH", 
                    'BELGIUM' : "ANHEUSER-BUSCH", 'ABINBEV' : "ANHEUSER-BUSCH", "ABI INSTITUCIONAL" :  "ANHEUSER-BUSCH",
                    "AB INBEV - BEER & LAGER RANGE" : "ANHEUSER-BUSCH", "ANHEUSER-BUSCH INBEV IMAGE" : "ANHEUSER-BUSCH",
                    
                    'MIKE S HARD' : "MIKES", 'MIKES HARD' : "MIKES", 'MIKES (REG)' : "MIKES", 
                    'MIKES HARDER LEMONADE' : "MIKES", 'MIKES HARDER' : "MIKES", 
                    
                    "Others" : "All Others",  "OTHERS" : "All Others", 
                    
                    'Cerveza Victoria'.upper() : "VICTORIA", 'CERVEZA VICTORIA' : "VICTORIA",
                    
                    'TIGER ()' : "TIGER", 'TIGER BEER' : "TIGER", 'HUDA ()' : "HUDA", 'TUBORG ()' :  "TUBORG", 
                    'IMPERIAL (CERVEZA)' : "IMPERIAL", 'CERVEZA IMPERIAL' : "IMPERIAL",
                    'KRONENBOURG 1664 BLANC ()' : "KRONENBOURG 1664 BLANC", 'LOA,CERVEZA' : "LOA", 
                    'EDELWEISS (CERVEZA)' : "EDELWEISS", 'CERVEZA BAJAMAR' :  "BAJAMAR", 
                    'CERVEZA CAMINO DEL SOL' : "CAMINO DEL SOL", 'CIA. CERVEZAS NACIONALES' : "CIA. NACIONALES",
                    'CERVEZA DORADA DRAFT' : "DORADA DRAFT", 'CERVEZA DORADA ICE' :  "DORADA ICE",
                    'CERVEZA GALLO' : "GALLO", 'GALLO CERVEZA' : "GALLO",
                    'CERVEZA TECATE' : "TECATE", 'CERVEZA BARENA' : "BARENA",
                    'CERVEZA GROLSCH' : "GROLSCH", 'CERVEZA MICHELOB' :  "MICHELOB",
                    'CERVEZA SALVA VIDA' : "SALVA VIDA", 'BALBOA CERVEZA' : "BALBOA", 'CERVEZA DORADA' : "DORADA",
                    'PRESIDENTE CERVEZA' : "PRESIDENTE", 'CERVEZA RREY' : "RREY", 
                    'CERVEZA SALTA CAUTIVA' : "SALTA CAUTIVA", 'BIERE LARUE ()' : "LARUE", 'HANOI ()' :  "HANOI",
                     'HANOI BEER' :  "HANOI",
                    'TRUC BACH ()' : "TRUC BACH", 'SAO VANG ()' : "SAO VANG", 'GUINNESS - BRAND BUILDING' : "GUINNESS",
                    'GUINNESS ZERO ALCOHOL' : "GUINNESS", 'GUINNESS HOP HOUSE 13 LAGER' : "GUINNESS", 
                    'GUINNESS STOUT' : "GUINNESS", 'GUINNESS DRAUGHT' : "GUINNESS", "GUINNESS NITROSURGE" :  "GUINNESS",
                    "GUINNESS DRINKS RANGE" : "GUINNESS", "GUINNESS - DRAUGHT STOUT" : "GUINNESS",
                    
                    'BREWDOG - PUNK IPA' : "BREWDOG" , 'BREWDOG - LOST LAGER' : "BREWDOG",
                    'TIMOTHY TAYLOR - HOPICAL STORM PALE' : "TIMOTHY TAYLOR",
                    'BEAVERTOWN BREWERY - CRAFT S & LAGER' : "BEAVERTOWN BREWERY", 
                    'BEAVERTOWN BREWERY - CRAFT BEERS & LAGER' : "BEAVERTOWN BREWERY",
                    
                    'COORS - FRESH COMEDY ON 4' : "MOLSON COORS", "COORS" : "MOLSON COORS",
                    'MOLSON COORS BREWING - MADRI EXCEPCIONAL' : "MOLSON COORS", 
                    'COORS BANQUET CERVEZA' : "MOLSON COORS", 
                    'COORS - LAGER' : "MOLSON COORS",
                    'JACK DANIELS/COCA COLA - TENNESSEE WHISKEY MIXED WITH COCA COLA' : "JACK DANIELS",
                    '1664 BLONDE' : "#1664", '8 6 BIERE' : "#86", 
                    'TOURTEL TWIST FRUITS & PLANTES SANS ALCOOL ALLEGE' : "TOURTEL", 
                    'TOURTEL TWIST BIERE SANS ALCOOL' : "TOURTEL", 'AFFLIGEM BIERE' : "AFFLIGEM", 
                    'GRIMBERGEN BRASSIN DE PRINTEMPS' : "GRIMBERGEN", 'ICHNUSA BIRRA' :  "ICHNUSA",
                    'ICHNUSA BIRRA I.U.' : "ICHNUSA", 'MESSINA BIRRA' : "MESSINA", 
                    'PERONI NASTRO AZZ.STILE CAPRI' : "PERONI NASTRO", 
                    'PERONI - NASTRO AZZURRO BEER' : "PERONI NASTRO",
                    'PERONI NASTRO AZZURRO BIRRA' : "PERONI NASTRO", 
                    'PERONI LIN.BIRRA' : "PERONI NASTRO",                     
                    'KOZEL BIRRA' : "KOZEL",
                    'GRIMBERGEN BIRRA' : "GRIMBERGEN", 'PORETTI 4 LUPPOLI BIRRA' :  "PORETTI",
                    'PORETTI LIN.BIRRA' : "PORETTI", 'CERES BIRRA' : "CERES", 'CERES PRO.BIRRA' : "CERES",
                    'CERES LIN.BIRRA' : "CERES",
                    'AFFLIGEM BLOND 0.0% ALCOHOLVRIJ SPECIAALBIER' :  "AFFLIGEM 0.0",
                    'AFFLIGEM BLOND SPECIAALBIER' : "AFFLIGEM", 'AFFLIGEM SPECIAALBIER ALGEMEEN' : "AFFLIGEM",
                    
                    'HERTOG JAN BIER PILS' : "HERTOG JAN", 'HERTOG JAN BOCKBIER SEIZOENSBIER' : "HERTOG JAN",
                    'HERTOG JAN GRAND PILSENER BIER PILS' : "HERTOG JAN", 
                    'TRIPEL KARMELIET SPECIAALBIER' : "TRIPEL KARMELIET", 
                    'KORDAAT PILSENER BIER' : "PILSENER", 'DUVEL SPECIAALBIEREN' : "DUVEL",
                    'KONINKLIJKE GROLSCH ENSCHEDE' : "GROLSCH", np.nan : "All Others",'POKER (REG)': "POKER", 
                    'AGUILA (REG)' : "AGUILA", "BEER CAPITOL DISTRIBUTING" : "BEER CAPITOL",
                    'ERDINGER - WHEAT BEER' : "ERDINGER", 'BAVARIA 8 6' : "BAVARIA 86", "BAVARIA 8.6 BIRRA" : "BAVARIA 86",
                    'JEVER FUN PILSENER ALKOHOLFREI SP' : "JEVER",
                    'JEVER FUN ALKOHOLFREI                 VB'  : "JEVER", 
                    'BITBURGER PILS' : "BITBURGER", 'BITBURGER 0,0 HERB ALKOHOLFREI' : "BITBURGER",
                    'RADEBERGER PILSNER                    VB' : "RADEBERGER",
                     'SCHOEFFERHOFER BIER RANGE             VB' : "SCHOEFFERHOFER",
                    "KROMBACHER PILS SP" : "KROMBACHER", "KROMBACHER PILS" : "KROMBACHER",
                    "SCHOEFFERHOFER BIER RANGE SP          VB" : "SCHOEFFERHOFER",
                    "CARLSBERG DANISH PILSNER" : "CARLSBERG", "Coors" : "COORS (Heineken Parent)", 
                    "Coors".upper() : "COORS (Heineken Parent)", "Coors Light" : "COORS LIGHT (Heineken Parent)",
                    "Coors Light".upper() : "COORS LIGHT (Heineken Parent)", "ASAHI NAMA BEER" : "ASAHI",
                    "ASAHI SUPER DRY" : "ASAHI", "CLEAR ASAHI": "ASAHI", "KIRIN HOME TAP" : "KIRIN", 
                    "KIRIN HONKIRIN" : "KIRIN", "PILSEN EXTRA" : "PILSEN", "AMSTEL BIER" : "AMSTEL",
                    "BUDWEISER BREW NO 66" : "BUDWEISER", "BRAHMA FRUTOS ROJOS" : "BRAHMA", 
                    "BRAHMA SUBZERO" : "BRAHMA", "BRAHMITA POMELO" : "BRAHMA", 'KF' : "KINGFISHER",
                    
                   }


df["Brand_standarization"]  = df["Brand_standarization"].replace(brand_dictionary)

### Focused Brand Competitors

In [ ]:
## Focus brand for the market:  It woudl be constant throughout the year

focus_brands = ['HEINEKEN', "HEINEKEN SILVER", "HEINEKEN 0.0", "AMSTEL", "AMSTEL ULTRA", 
               "AMSTEL ULTRA SELTZER", "AMSTEL LAGER", "AMSTEL MALTA", "AMSTEL LIGHT"]

df["Focused_Brand_Competitors"] = df["Brand_standarization"].copy()

# Assign most values in the 'Top_Brand_Competitors_and_ABI' column as 'Other' except for specific values
df['Focused_Brand_Competitors'] = np.where(df['Focused_Brand_Competitors'].isin(focus_brands), 
                                              df['Focused_Brand_Competitors'], 'All Others')
df['Focused_Brand_Competitors'].unique()

### Media Summary

In [14]:
# Standarize Media Summary column

"""
    Media summary is focused on getting feature values into five differnt media types i.e. 
     "TV", "Radio", "Print", "Print", "OOH", "Digital". This provides high level summary
     of main media channels or types. 

"""

df["Media_Summary"] = df["Media_type"].copy() 

tv = ['Tv', 'Cable TV', 'PAY TV', 'Span Lang Net TV', 'TVPagada', 'CABLE', 'LOCAL TV', 'Network TV', 'Open TV', 'OPEN TV', 
      'OTV', 'Television', 'Advertising Broadcasts', 'PAID TV (CABLE)',
       'PTV', 'Spot TV', 'TELEVISION', 'TV', 'tv', 'TV ABIERTA', 'TV Abierta', 'TV ABIERTA ', 'TV CABLE',
      'TV Capital', 'TV Interior', 'TV PAGADA', 'TV Cable', 'Syndication', 'Pay TV', 'TV PAGA', 'TV LOCAL']
radio = ['Natl Spot Radio', 'Radio AM', 'Local Radio', 'Network Radio', 'RADIO', 'Radio', 'Radio FM', 'Radios', 'RD',
        'RADIO REGIONAL']
print_ = ['Diarios Capital', 'NEWSPAPER', 'NEWSPAPERS', 'Newspapers', 'PRESS', 'REVISTAS', 'B-to-B Magazines', 'DIARIOS', 
          'Diarios Interior', 'Hispanic Newspapers', 'Local Magazines', 'Local Media', 'MAGAZINE', 'Magazine', 'Magazines',
          'METRO', 'Newspaper', 'NP', 'Prensa', 'PRENSA', 'Prensa Escrita', 'PRINT', 'Revista', 'Revistas', 'Suplemento',
         'Press', 'Door Drops', 'PRESSE', 'Trade Journals', 'Trade press', 'REVISTA', 'DIARIO', 'PRINTS', 'MAGAZINES',
         'MG']
ooh = ['Outdoor', 'EXTERIORES', 'OOH', 'OUT OF HOME', 'VIA PUBLICA', 'Vía Pública', 'Cinema', 'CINEMA', 'VP', '(D)OOH', 
      'Poster', 'Transport Media', 'OutOfHome', 'Out Of Home', 'Ambient Media', 'OUTROS', 'Out of Home']
digital = ['INTERNET', 'Internet', 'Internet - Display', 'Internet - Search', 'Digital', 'Programmatic', 
           'DIGITAL', 'META', 'Mobile App', 'Mobile Web', 'Mobile Web Video', 'Online Video', 'Paid Social', 'Social',
           'Youtube', 'AVOD', 'Search', 'IN-HOME ADVERTISING', 'Internet Display Ag.', 'Mobile', 'Desktop', 'Retail Media',
          'DigitalMed', 'Online display', 'Direct Mail', 'Internet log']

df['Media_Summary'] = df['Media_Summary'].replace(tv, "TV")
df['Media_Summary'] = df['Media_Summary'].replace(radio, "Radio")
df['Media_Summary'] = df['Media_Summary'].replace(print_, "Print")
df['Media_Summary'] = df['Media_Summary'].replace(ooh, "OOH")
df['Media_Summary'] = df['Media_Summary'].replace(digital, "Digital")

df['Media_Summary'].unique()

array(['OOH', 'Digital', 'TV', 'Radio', 'Print', 'TV ABERTA', 'JORNAL',
       'TV ASSINATURA', 'TV MERCHANDISING', 'Other', 'RADIO ', 'METRO ',
       'OOH ', 'RADIO REGIONAL ', 'PRENSA ', 'Ooh', 'Ptv', 'Otv',
       'Display', 'Dailies', 'Trade magazines', 'Online'], dtype=object)

### Media Type (Standardized)

In [15]:
# Media Type (Standardized)
df["Media_type_standarized"] = df["Media_type"].copy() 

cable_tv = ['Cable TV', 'CABLE', 'PAID TV (CABLE)', 'TV CABLE', 'TV Cable']
open_tv = ['OPEN TV', 'OTV', 'Open TV']
paid_tv = ['PAY TV', 'TVPagada', 'PTV', 'TV PAGADA', 'PAID TV (CABLE)', 'Pay TV', 'TV PAGA']
free_tv = ['TV ABIERTA', 'TV Abierta', 'TV ABIERTA ']
spot_tv = [ 'Spot TV']
tv = ['TELEVISION', 'TV', 'tv', 'Tv', 'Advertising Broadcasts', 'Television']
am_radio = ['Radio AM']
fm_radio = [ 'Radio FM']
local_radio = [ 'Local Radio']
natl_spot_radio = ['Natl Spot Radio']
network_radio = [ 'Network Radio']
radio = ['RADIO', 'Radio', 'Radios', 'RD']
cinema = ['Cinema', 'CINEMA']
ooh = ['Outdoor', 'EXTERIORES', 'OOH', 'OUT OF HOME', 'VIA PUBLICA', 'Vía Pública',  'VP', '(D)OOH', 'Poster', 
        'OutOfHome', 'Out Of Home', 'OUTROS']
print_ = ['PRESS', 'Prensa', 'PRENSA', 'Prensa Escrita', 'PRINT', 'Press', 'PRESSE', 'DIARIO', 'PRINTS']
newspapers = ['Diarios Capital', 'NEWSPAPER', 'NEWSPAPERS', 'Newspapers', 'Diarios Interior',
              'METRO', 'Newspaper', 'NP', 'Suplemento', 'DIARIOS', 'Trade press']
magazines = ['REVISTAS', 'MAGAZINE', 'Magazine', 'Magazines', 'Revista', 
             'Revistas', 'Door Drops', 'REVISTA', 'MAGAZINES']
digital = ['Digital', 'DIGITAL', 'Search', 'DigitalMed', 'MG']
syndication = ['Syndication']

avod = ['AVOD']
internet = ['Internet', 'INTERNET', 'Internet log']
internet_display = ['Internet - Display', 'Internet Display Ag.']
internet_search = ['Internet - Search']
meta = ['META']

mobile = ['Mobile']
desktop = ['Desktop']
mobile_app = [ 'Mobile App']    
mobile_web = ['Mobile Web']
mobile_web_video = ['Mobile Web Video']
online_video = ['Online Video']
paid_social = ['Paid Social']
programmatic = ['Programmatic']
social = ['Social']
yt = ['Youtube']
search = ['Search']
bb_magazines = ['B-to-B Magazines']
trade_journals = ['Trade Journals']
hispanic_newspapers = ['Hispanic Newspapers']
local_magazines = ['Local Magazines'] 
local_media = ['Local Media']
local_tv = ['LOCAL TV'] 
span_lang_net_tv = ['Span Lang Net TV']
tv_capital = ['TV Capital']
tv_interior = ['TV Interior']
radio_regional = ['RADIO REGIONAL']
direct_mail = ['Direct Mail']
in_home_adv = ['IN-HOME ADVERTISING']
transport_mdeia = ['Transport Media']
ambient_media = ['Ambient Media']
retail_media = ['Retail Media']

df['Media_type_standarized'] = df['Media_type_standarized'].replace(trade_journals, "Trade Journals")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(retail_media, "Retail Media")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(ambient_media, "Ambient Media")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(transport_mdeia, "Transport Media")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(in_home_adv, "In-Home Advertising")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(direct_mail, "Direct Mail")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(radio_regional, "Regional Radio")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(avod, "AVOD")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(internet, "Internet")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(internet_display, "Internet Display")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(internet_search, "Internet Search")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(meta, "Meta")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(mobile_app, "Mobile App")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(mobile, "Mobile")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(desktop, "Desktop")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(mobile_web, "Mobile Web")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(mobile_web_video, "Mobile Web Video")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(online_video, "Online Video")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(paid_social, "Paid Social")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(programmatic, "Programmatic")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(social, "Social")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(yt, "Youtube")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(search, "Search")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(bb_magazines, "B-to-B Magazines")

df['Media_type_standarized'] = df['Media_type_standarized'].replace(hispanic_newspapers, "Hispanic Newspapers")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(local_magazines, "Local Magazines")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(local_media, "Local Media")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(local_tv, "Local TV")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(span_lang_net_tv, "Span Lang Net TV")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(syndication, "Syndication")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(tv_capital, "TV Capital")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(tv_interior, "TV Interior")

df['Media_type_standarized'] = df['Media_type_standarized'].replace(cable_tv, "Cable TV")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(open_tv, "Open TV")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(paid_tv, "Paid TV")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(free_tv, "Free TV")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(spot_tv, "Spot TV")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(tv, "TV")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(am_radio, "AM Radio")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(fm_radio, "FM Radio")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(local_radio, "Local Radio")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(natl_spot_radio, "Natl Spot Radio")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(network_radio, "Network Radio")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(radio, "Radio")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(cinema, "Cinema")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(print_, "Print")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(newspapers, "Newspapers")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(magazines, "Magazines")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(ooh, "OOH")
df['Media_type_standarized'] = df['Media_type_standarized'].replace(digital, "Digital")

df['Media_type_standarized'].unique()

array(['OOH', 'Internet', 'TV Interior', 'FM Radio', 'Cable TV',
       'TV Capital', 'Newspapers', 'Magazines', 'AM Radio', 'Print',
       'In-Home Advertising', 'TV', 'Cinema', 'Radio', 'Meta', 'Youtube',
       'Programmatic', 'Local Media', 'Digital', 'Paid TV', 'TV ABERTA',
       'JORNAL', 'TV ASSINATURA', 'TV MERCHANDISING', 'Open TV', 'Social',
       'Other', 'RADIO ', 'Free TV', 'METRO ', 'OOH ', 'RADIO REGIONAL ',
       'PRENSA ', 'Ooh', 'Ptv', 'Otv', 'Display', 'Local TV',
       'Out of Home', 'Dailies', 'Trade magazines', 'Online',
       'Direct Mail', 'Local Radio', 'Paid Social', 'Mobile App',
       'Mobile Web', 'AVOD', 'Local Magazines', 'Online Video',
       'Natl Spot Radio', 'Spot TV', 'B-to-B Magazines',
       'Internet Display', 'Network TV', 'Internet Search',
       'Mobile Web Video', 'Span Lang Net TV', 'Network Radio',
       'Hispanic Newspapers', 'Syndication'], dtype=object)

### Month (Number)

In [16]:
# Month (Number)
df["Month_number"] = df["Month"].copy()
df["Month_number"].unique()

array([11, 2, 4, 5, 6, 7, 8, 9, 10, 12, 3, 1, 'June', 'March', 'April',
       'July', 'August', 'September', 'October', 'November', 'December',
       'May', 'January', 'February', 'Octuber', 'JAN', 'FEV', 'MAR',
       'ABR', 'MAI', 'JUN', 'JUL', 'AGO', 'SET', 'OUT', 'NOV', 'DEZ',
       'ENERO ', 'FEBRERO ', 'MARZO ', 'MARZO', 'ABRIL ', 'MAYO', 'JUNIO',
       'JULIO', 'AGOSTO', 'SEPTIEMBRE', 'OCTUBRE', 'NOVIEMBRE',
       'DICIEMBRE', 'ENERO', 'FEBRERO', 'ABRIL', 'MAYO ', 'JUNIO ',
       'Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio',
       'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre',
       'enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio', 'julio',
       'septiembre', 'agosto', 'noviembre', 'diciembre', 'octubre', 'FEB',
       'APR', 'MAY', 'AUG', 'SEP', 'OCT', 'DIC', 'Oct', 'Nov', 'Dic',
       'Jan', 'Feb', 'Mar', 'Apr', 'Jun', 'Jul', 'Aug', 'Sep', 'Dec',
       'july', 'august', 'Ene', 'Abr', 'mar', 'Ago', 'JANUARY',
       'FEBRUARY', '

In [17]:
jan_month = ["Jan", "JAN", "JANUARY", "January", "Ene", "enero", "Enero", "January 2023", 'JANUARY ', "ENERO", "ENE",
            'ENERO ']
feb_month = ["FEB", "Feb", "Febrero", "febrero", "FEBRUARY", "February", "February 2023", "FEV", 'FEBRUARY ', "FEBRERO",
            'FEBRERO ']
march_month = ["MAR", "Mar", "mar", "MARCH", "March", "Marzo", "marzo", "March 2023", 'MARCH ', "MARZO", 'MARZO ']
april_month = ["April 2023", "ABR", "Abr", "Abril","abril", "ABRIL", "Apr", "APR", "April", "APRIL", 'APRIL ',
              'ABRIL ']
may_month = ["MAY", "May", "Mayo", "mayo", "MAYO", "May 2023", "MAI", 'MAY ', 'MAYO ']
june_month = ["Jun", "June", "JUNIO", "Junio", "junio", "JUN", "JUNE", "JUn", "June 2023", 'JUNE ', 'JUNIO ']
july_month = ["Julio", "julio", "Jul", "JUL", "July", "JULIO", "JULY", 'JULY ', 'july']
aug_month = ["AGO", "agosto", "AUG", "Ago", "Aug", "August", "AGOSTO", "AUGUST", "Agosto", 'AUGUST ', 'august']
sep_month = ["SEP", "Sep", "septiembre", "September", "Septiembre", "SETIEMBRE", "SEPTEMBER", "SET", "SEPTIEMBRE", 
            "Sept",'SEPTEMBER ']
oct_month = ["October", "OCTOBER", "OCT", "Oct", 'OCTOBER ', 'Octuber', 'OUT', 'OCTUBRE', 'Octubre', 'octubre']
nov_month = ["November", "NOVEMBER", "NOV", "Nov", 'NOVEMBER ', 'NOVIEMBRE', 'Noviembre', 'noviembre']
dec_month = ["December", "DECEMBER", "DEC", "Dec", 'DEZ',  'DICIEMBRE', 'Diciembre', 'diciembre', 'DIC', 'Dic']


df.loc[df['Month_number'].isin(jan_month), 'Month_number'] = 1
df.loc[df['Month_number'].isin(feb_month), 'Month_number'] = 2
df.loc[df['Month_number'].isin(march_month), 'Month_number'] = 3
df.loc[df['Month_number'].isin(april_month), 'Month_number'] = 4
df.loc[df['Month_number'].isin(may_month), 'Month_number'] = 5
df.loc[df['Month_number'].isin(june_month), 'Month_number'] = 6
df.loc[df['Month_number'].isin(july_month), 'Month_number'] = 7
df.loc[df['Month_number'].isin(aug_month), 'Month_number'] = 8
df.loc[df['Month_number'].isin(sep_month), 'Month_number'] = 9
df.loc[df['Month_number'].isin(oct_month), 'Month_number'] = 10
df.loc[df['Month_number'].isin(nov_month), 'Month_number'] = 11
df.loc[df['Month_number'].isin(dec_month), 'Month_number'] = 12


df.loc[df['Month_number'] == pd.Timestamp('2023-01-01 00:00:00'), 
       'Month_number'] = pd.to_datetime('2023-01-01 00:00:00').month

df.loc[df['Month_number'] == pd.Timestamp('2023-02-01 00:00:00'), 
       'Month_number'] = pd.to_datetime('2023-02-01 00:00:00').month

df.loc[df['Month_number'] == pd.Timestamp('2023-03-01 00:00:00'),
       'Month_number'] = pd.to_datetime('2023-03-01 00:00:00').month

df.loc[df['Month_number'] == pd.Timestamp('2023-04-01 00:00:00'), 
       'Month_number'] = pd.to_datetime('2023-04-01 00:00:00').month

df.loc[df['Month_number'] == pd.Timestamp('2023-05-01 00:00:00'), 
       'Month_number'] = pd.to_datetime('2023-05-01 00:00:00').month

df.loc[df['Month_number'] == pd.Timestamp('2023-06-01 00:00:00'), 
       'Month_number'] = pd.to_datetime('2023-06-01 00:00:00').month

df.loc[df['Month_number'] == pd.Timestamp('2023-07-01 00:00:00'), 
       'Month_number'] = pd.to_datetime('2023-07-01 00:00:00').month

df.loc[df['Month_number'] == pd.Timestamp('2023-08-01 00:00:00'), 
       'Month_number'] = pd.to_datetime('2023-08-01 00:00:00').month

df.loc[df['Month_number'] == pd.Timestamp('2023-09-01 00:00:00'), 
       'Month_number'] = pd.to_datetime('2023-09-01 00:00:00').month

df.loc[df['Month_number'] == pd.Timestamp('2023-10-01 00:00:00'), 
       'Month_number'] = pd.to_datetime('2023-10-01 00:00:00').month

df.loc[df['Month_number'] == pd.Timestamp('2023-11-01 00:00:00'), 
       'Month_number'] = pd.to_datetime('2023-11-01 00:00:00').month 

df.loc[df['Month_number'] == pd.Timestamp('2023-12-01 00:00:00'), 
       'Month_number'] = pd.to_datetime('2023-12-01 00:00:00').month

df["Month_number"].unique()

array([11, 2, 4, 5, 6, 7, 8, 9, 10, 12, 3, 1], dtype=object)

### Month (Standardized)

In [18]:
df['Month_standarization'] = pd.to_datetime(df['Year'].astype(str) + '-' + df['Month_number'].astype(str))

#### Creating Dataframes Market Wise

In [19]:
# Its requried to create number of filtered dataframes for each country to use 
# pivot table to find top competetor, top 10 markets

df_argentina = df[df['Country'] == 'Argentina']
df_bolivia = df[df['Country'] == 'Bolivia']
df_brazil = df[df['Country'] == 'Brazil']
df_canada = df[df['Country'] == 'Canada']
df_chile = df[df['Country'] == 'Chile']
df_china = df[df['Country'] == 'China']
df_colombia = df[df['Country'] == 'Colombia']
df_dom_rep = df[df['Country'] == 'Dominican Republic']
df_ecuador = df[df['Country'] == 'Ecuador']
df_el_salvador = df[df['Country'] == 'El Salvador']
df_guatemala = df[df['Country'] == 'Guatemala']
df_honduras = df[df['Country'] == 'Honduras']
df_india = df[df['Country'] == 'India']
df_japan = df[df['Country'] == 'Japan']
df_mexico = df[df['Country'] == 'Mexico']
df_panama = df[df['Country'] == 'Panama']
df_paraguay = df[df['Country'] == 'Paraguay']
df_peru = df[df['Country'] == 'Peru']
df_south_africa = df[df['Country'] == 'South Africa']
df_south_korea = df[df['Country'] == 'South Korea']
df_uruguay = df[df['Country'] == 'Uruguay']
df_us = df[df['Country'] == 'US']
df_uk = df[df['Country'] == 'UK']
df_vietnam = df[df['Country'] == 'Vietnam']

df_be = df[df['Country'] == 'Belgium']
df_fr = df[df['Country'] == 'France']
df_de = df[df['Country'] == 'Germany']
df_it = df[df['Country'] == 'Italy']
df_nl = df[df['Country'] == 'Netherlands']
df_ie = df[df['Country'] == 'Ireland']


markets = [df_argentina, df_bolivia, df_brazil, df_canada, df_chile, df_china, df_colombia, df_dom_rep, df_ecuador, 
           df_el_salvador, df_guatemala, df_honduras, df_india, df_japan, df_mexico, df_panama, df_paraguay, df_peru, 
           df_south_africa, df_south_korea, df_uruguay, df_us, df_uk, df_vietnam, df_be, df_fr, df_de, df_it, df_nl,
          df_ie]

print("The Number of markets for Competitive Analysis: {}\n".format(len(markets)))

for market in markets:
    print("The number of data points for country: {}\t\t:{}".format(market["Country"].unique(), market.shape[0]))

The Number of markets for Competitive Analysis: 30

The number of data points for country: ['Argentina']		:2645
The number of data points for country: ['Bolivia']		:119
The number of data points for country: ['Brazil']		:1399
The number of data points for country: ['Canada']		:2213
The number of data points for country: ['Chile']		:566
The number of data points for country: ['China']		:592
The number of data points for country: ['Colombia']		:1372
The number of data points for country: ['Dominican Republic']		:210638
The number of data points for country: ['Ecuador']		:960
The number of data points for country: ['El Salvador']		:28889
The number of data points for country: ['Guatemala']		:177803
The number of data points for country: ['Honduras']		:11399
The number of data points for country: ['India']		:476
The number of data points for country: ['Japan']		:361
The number of data points for country: ['Mexico']		:31501
The number of data points for country: ['Panama']		:282
The number 

### Parent Competitors (Pivots)

In [20]:
## Excluding Other Alcohol category for Competitive analysis of Beer

## Operation steps : 
# 1. filerter datframe by excluding Other Alcohol category
# 2. Use groupby fuction by adv. parent with aggregation with percentage applying lambda fucntion
# 3. Sort values in descending
# 4. Spend_US_Currency column in pivot table would be in percentage

## Function to find top 5 parent competitor in a market in beer category given their ad spend market share in above 5%

def parent_competitors_std(df):
    competitors_df = df[(df['Category_Standardized'] != 'Other Alcohol')]
    competitors_df_gb = competitors_df.groupby([ 'Advertiser_parent_standarized']).agg({'Spend_US_Currency': lambda x: 
                                                                                      round(x.sum()/competitors_df["Spend_US_Currency"].sum()*100,
                                                                                            2)}).sort_values('Spend_US_Currency', 
                                                                                                             ascending = False)
    competitors_df_gb = competitors_df_gb.reset_index()
    prefixes = ['HEINEKEN', 'ABI', "AMSTEL", "CCU (Heineken Parent", "CCC (Heineken Parent)"]
    competitors_df_gb = (competitors_df_gb[(competitors_df_gb["Spend_US_Currency"] >= 
                                            5) | (competitors_df_gb["Advertiser_parent_standarized"].str.startswith(tuple(prefixes)))])
    
    exceptions = competitors_df_gb["Advertiser_parent_standarized"].tolist()
    # Add new column and copy data from adv parent std; except exceptions - rest of the company are as All Others
    df['Parent_competitor'] = df['Advertiser_parent_standarized'].copy()
    df['Parent_competitor'] = np.where(df['Parent_competitor'].isin(exceptions), 
                                                   df['Parent_competitor'], "All Others")
    print("\n")
    return df['Parent_competitor'].unique()

# Apply The function to all the market dfs
for market in markets:
    print("{}:{}".format(market.Country.unique(), parent_competitors_std(market)))



['Argentina']:['All Others' 'HEINEKEN' 'ABI']


['Bolivia']:['NUR' 'ABI' 'BBO']


['Brazil']:['ABI' 'HEINEKEN' 'PETROPOLIS' 'All Others']


['Canada']:['MOLSON COORS' 'HEINEKEN' 'All Others' 'SLEEMAN' 'ABI']


['Chile']:['CCU' 'ABI' 'All Others']


['China']:['ABI' 'All Others' 'CARLSBERG' 'HEINEKEN' 'TSINGTAO' 'YANJING']


['Colombia']:['All Others' 'IND LICORERA CALDAS' 'HEINEKEN' 'DIAGEO'
 'PERNOD RICARD COLOMB' 'FAB LIC ANTIOQUIA' 'IND LICORES VALLE'
 'WILLIAM GRANT & SONS']


['Dominican Republic']:['ABI' 'All Others' 'HEINEKEN']


['Ecuador']:['ABI' 'HEINEKEN' 'All Others']


['El Salvador']:['ABI' 'All Others' 'HEINEKEN' 'CADEJO' 'CERVECERIA CENTROAMERICANA'
 'BIRRIA FEST']


['Guatemala']:['HEINEKEN' 'All Others' 'ABI']


['Honduras']:['CERVECERIA HONDUREÃƑÂ€˜A' 'CERVECERIA HONDUREÃ‘A' 'DISTRIBUIDORA SOLIS'
 'HEINEKEN' 'CERVEZA DORADA ICE' 'All Others']


['India']:['UNITED BREWERIES' 'ABI' 'CARLSBERG'
 'PERNOD RICARD INDIA PRIVATE LIMITED' 'BACARDI MARTINI INDIA'
 'All Other

### Top 10 Brands by Market

In [21]:
def top_10_brands(df):
    beer_df = df[(df['Category_Standardized'] != 'Other Alcohol')]
    top10_df = beer_df.groupby([ 'Brand_standarization']).agg({'Spend_US_Currency': 
                                                                            "sum"}).sort_values('Spend_US_Currency',
                                                                                                ascending = False).head(10)
    top10_df = top10_df.reset_index()
        
    exceptions = top10_df["Brand_standarization"].tolist()
    # Add new column and copy data from adv parent std; except exceptions - rest of the company are as All Others
    df['Top10_Brands_by_Market'] = df['Brand_standarization'].copy()
    df['Top10_Brands_by_Market'] = np.where(df['Top10_Brands_by_Market'].isin(exceptions), 
                                                   df['Top10_Brands_by_Market'], "All Others")
    
  
    print("\n")
    return df['Top10_Brands_by_Market'].unique()

# Apply The function to all the market dfs
for market in markets:
    print("{}:{}".format(market.Country.unique(), top_10_brands(market)))



['Argentina']:['All Others' 'HEINEKEN' 'IMPERIAL' 'NORTE' 'SALTA' 'SCHNEIDER' 'ANDES'
 'BRAHMA' 'CORONA' 'QUILMES' 'STELLA ARTOIS']


['Bolivia']:['BURGUESA' 'HUARI' 'PACENA' 'GOLDEN' 'AMSTEL' 'CONTI']


['Brazil']:['BRAHMA' 'SPATEN' 'CORONA' 'BUDWEISER ZERO' 'SKOL' 'All Others'
 'HEINEKEN' 'AMSTEL' 'HEINEKEN 0.0' 'EISENBAHN' 'ITAIPAVA']


['Canada']:['COORS LIGHT (Heineken Parent)' 'HEINEKEN' 'All Others'
 'MOLSON PARTNERSHIP' 'MILLER LITE' 'STELLA ARTOIS' 'BUD LIGHT' 'CORONA'
 'MICHELOB ULTRA' 'BUDWEISER' 'SLEEMAN CLEAR 2.0']


['Chile']:['CRISTAL' 'ESCUDO' 'All Others' 'ROYAL GUARD' 'SOL' 'STONES' 'HEINEKEN'
 'AUSTRAL' 'COORS (Heineken Parent)' 'BUDWEISER' 'MILLER']


['China']:['BUDWEISER' 'All Others' 'HARBIN' 'BLUE GIRL' 'TUBORG' 'WUSU' 'HEINEKEN'
 'SNOW' 'TSINGTAO' 'YANJING']


['Colombia']:['All Others' 'VIEJO DE CALDAS' 'MEDELLIN' 'JACK DANIELS' 'HEINEKEN'
 'BLANCO DEL VALLE' 'ANTIOQUENO' 'ANDINA' 'CHIVAS REGAL' 'BUCHANANS'
 'MONKEY SHOULDER']


['Dominican Republic']:['All 

### Merging all the files back to main datafram

In [22]:
df_ = pd.DataFrame()
for market in markets:
    df_ = df_.append(market)

### Global Focus Brand

In [23]:
df_['Focused_Global_Brand'] = df_['Brand_standarization'].copy()
global_brands = ["AMSTEL", 'AMSTEL LAGER', 'AMSTEL LIGHT', 'AMSTEL MALTA', 'AMSTEL ULTRA', 'AMSTEL ULTRA SELTZER', 
                 'HEINEKEN',
                 'HEINEKEN 0.0', 'HEINEKEN SILVER']

 
df_['Focused_Global_Brand'] = np.where(df_['Focused_Global_Brand'].isin(global_brands), 
                                                   df_['Focused_Global_Brand'], "All Others")
df_['Focused_Global_Brand'].unique()

array(['All Others', 'AMSTEL', 'HEINEKEN', 'HEINEKEN 0.0', 'AMSTEL ULTRA',
       'HEINEKEN SILVER', 'AMSTEL ULTRA SELTZER', 'AMSTEL LAGER',
       'AMSTEL MALTA', 'AMSTEL LIGHT'], dtype=object)

### Global Parent

In [24]:
df_["Global_Parent"] = df_['Parent_competitor'].copy()

In [25]:
 ## NEED TO DISCUSS WITH BOB

df_.loc[df_["Global_Parent"] == "CCU SA (Heineken Parent)", "Global_Parent"] = "HEINEKEN"
df_.loc[df_["Global_Parent"] == "CCU (Heineken Parent)", "Global_Parent"] = "HEINEKEN"
df_.loc[df_["Global_Parent"] == "CR Beer (Heineken Parent)", "Global_Parent"] = "HEINEKEN"
df_.loc[df_["Global_Parent"] == "CCC (Heineken Parent)", "Global_Parent"] = "HEINEKEN"
df_.loc[df_["Global_Parent"] == "VBL (Heineken Parent)", "Global_Parent"] = "HEINEKEN"
df_["Global_Parent"].unique()

array(['All Others', 'HEINEKEN', 'ABI', 'NUR', 'BBO', 'PETROPOLIS',
       'MOLSON COORS', 'SLEEMAN', 'CCU', 'CARLSBERG', 'TSINGTAO',
       'YANJING', 'IND LICORERA CALDAS', 'DIAGEO', 'PERNOD RICARD COLOMB',
       'FAB LIC ANTIOQUIA', 'IND LICORES VALLE', 'WILLIAM GRANT & SONS',
       'CADEJO', 'CERVECERIA CENTROAMERICANA', 'BIRRIA FEST',
       'CERVECERIA HONDUREÃƑÂ€˜A', 'CERVECERIA HONDUREÃ‘A',
       'DISTRIBUIDORA SOLIS', 'CERVEZA DORADA ICE', 'UNITED BREWERIES',
       'PERNOD RICARD INDIA PRIVATE LIMITED', 'BACARDI MARTINI INDIA',
       'UNITED SPIRITS LTD', 'KIRIN', 'SAPPORO', 'ASAHI', 'SUNTORY',
       'FEDURO', 'EMBOTELLADORA CENTRAL', 'HITE JINRO', 'LOTTE BG',
       'REGIONAL SUR', 'PONTYN SA', 'ABITA BREWING CO', 'BOSTON BEER',
       'CONSTELLATION', 'BREWDOG', 'BEAVERTOWN BREWERY', 'SABECO',
       'HABECO', 'DUVEL', 'DUVEL MOORTGAT NEDERLAND AMSTERDAM',
       'KONINKLIJKE GROLSCH ENSCHEDE', 'TEXELSE BIERBROUWERIJ OUDESCHILD'],
      dtype=object)

### ABI Parent SOS

In [26]:
df_['ABI_Parent_SOS'] = df_["Global_Parent"].copy()
abi = ['ABI']
df_['ABI_Parent_SOS'] = np.where(df_['ABI_Parent_SOS'].isin(abi), df_['ABI_Parent_SOS'], "All Others")
df_['ABI_Parent_SOS'].unique()

array(['All Others', 'ABI'], dtype=object)

### Heineken Brand SOS

In [27]:
df_['Heineken_Brand_SOS'] = df_['Focused_Global_Brand'].copy()
df_['Heineken_Brand_SOS'].value_counts()

All Others              486417
AMSTEL                   12786
HEINEKEN                  5692
HEINEKEN SILVER           1074
HEINEKEN 0.0               651
AMSTEL ULTRA SELTZER       496
AMSTEL ULTRA                39
AMSTEL LAGER                30
AMSTEL MALTA                12
AMSTEL LIGHT                12
Name: Heineken_Brand_SOS, dtype: int64

In [28]:
df_['Heineken_Brand_SOS'] = df_['Heineken_Brand_SOS'].str.contains('HEINEKEN')
df_.loc[df_['Heineken_Brand_SOS'] == True, 'Heineken_Brand_SOS'] = 'HEINEKEN'
df_.loc[df_['Heineken_Brand_SOS'] == False, 'Heineken_Brand_SOS'] = 'All Others'
df_['Heineken_Brand_SOS'].value_counts()

All Others    499792
HEINEKEN        7417
Name: Heineken_Brand_SOS, dtype: int64

### Amstel Brand SOS

In [29]:
df_["Amstel_Brand_SOS"] = df_['Focused_Global_Brand'].copy()
df_["Amstel_Brand_SOS"].value_counts()

All Others              486417
AMSTEL                   12786
HEINEKEN                  5692
HEINEKEN SILVER           1074
HEINEKEN 0.0               651
AMSTEL ULTRA SELTZER       496
AMSTEL ULTRA                39
AMSTEL LAGER                30
AMSTEL MALTA                12
AMSTEL LIGHT                12
Name: Amstel_Brand_SOS, dtype: int64

In [30]:
df_["Amstel_Brand_SOS"] = df_["Amstel_Brand_SOS"].str.contains("AMSTEL")
df_.loc[df_["Amstel_Brand_SOS"] == True, "Amstel_Brand_SOS"] = "AMSTEL"
df_.loc[df_["Amstel_Brand_SOS"] == False, "Amstel_Brand_SOS"] = 'All Others'
df_["Amstel_Brand_SOS"].value_counts()

All Others    493834
AMSTEL         13375
Name: Amstel_Brand_SOS, dtype: int64

### Digital / Traditional

In [31]:
df_["Digital_Traditional"] = df_["Media_Summary"].copy()
df_["Digital_Traditional"].value_counts()

Radio               320971
TV                  122531
Digital              42702
OOH                  16219
Print                 2486
Display                833
Ooh                    313
TV ABERTA              308
Other                  216
Ptv                    183
Online                 100
RADIO                   82
Otv                     65
TV ASSINATURA           60
TV MERCHANDISING        38
METRO                   26
OOH                     24
Trade magazines         19
Dailies                 16
JORNAL                  13
RADIO REGIONAL           3
PRENSA                   1
Name: Digital_Traditional, dtype: int64

In [32]:
## Except digital media summrary, rest catagorize as Traditional
digital = ["Digital"]
df_["Digital_Traditional"] = np.where(df_["Digital_Traditional"].isin(digital), df_["Digital_Traditional"], "Traditional") 
df_["Digital_Traditional"].value_counts()

Traditional    464507
Digital         42702
Name: Digital_Traditional, dtype: int64

### Heineken Parent SOS

In [33]:
df_["Heineken_Parent_SOS"] = df_["Global_Parent"].copy()
#df_["Heineken_Parent_SOS"].value_counts()
exception = ["HEINEKEN"]
df_["Heineken_Parent_SOS"] = np.where(df_["Heineken_Parent_SOS"].isin(exception), df_["Heineken_Parent_SOS"], "All Others")
df_["Heineken_Parent_SOS"].value_counts()

All Others    337062
HEINEKEN      170147
Name: Heineken_Parent_SOS, dtype: int64

### Market Group

In [34]:
# Groups as per markets
group_I = ['Argentina', 'Brazil', 'Canada', 'China', 'Colombia', 'India', 'Mexico', 'South Africa', 'South Korea', 'US',
          "Belgium", "France", "Germany", "Ireland", "Italy", "Netherlands", "UK"]
group_II = ["Bolivia", "Chile", 'Dominican Republic', 'Ecuador', 'El Salvador', 'Guatemala', 'Honduras', 'Japan', 'Panama',
            'Paraguay', 'Peru', 'Uruguay', "Ukraine", 'Vietnam', ]

df_["Market_Group"] = df_["Country"].copy()

# Catgorize markets in two groups
categorize = lambda x: "Group I" if x in group_I else "Group II"
df_["Market_Group"] = df_["Market_Group"].apply(categorize)
df_["Market_Group"].value_counts()

Group II    434266
Group I      72943
Name: Market_Group, dtype: int64

### Brand Rank by Country

In [35]:
brand_beer = df_.loc[df_["Category_Standardized"] != "Other Alcohol"]
beer_pivot = brand_beer.groupby(["Country", "Brand_standarization"]).agg({'Spend_US_Currency': "sum"})
beer_pivot = beer_pivot.reset_index()

beer_pivot_all = pd.DataFrame()
for country in beer_pivot.Country.unique():
    beer_pivot_each = beer_pivot.loc[beer_pivot["Country"] == country]
    beer_pivot_each_ = beer_pivot_each.sort_values("Spend_US_Currency", ascending = False)
    beer_pivot_each_ = beer_pivot_each_.reset_index()
    beer_pivot_each_.drop('index', axis = 1, inplace = True)

    beer_pivot_each_ = beer_pivot_each_.assign(Brand_Rank_by_Country = 
                                           beer_pivot_each_["Spend_US_Currency"].rank(ascending = False))
    
    beer_pivot_all = beer_pivot_all.append(beer_pivot_each_)

beer_pivot_all["Brand_Rank_by_Country"] = beer_pivot_all["Brand_Rank_by_Country"].astype(int)
beer_pivot_all["Brand_Rank_by_Country"] = beer_pivot_all["Brand_Rank_by_Country"].apply(lambda x: 11 if x > 10 else x)
#beer_pivot_all

# Craete another column by combining country & Brand standarised
beer_pivot_all["Country_and_Brand"] = beer_pivot_all["Country"] + "_" + beer_pivot_all["Brand_standarization"]

# Only considering top 10 brand ranks rest can be catagorize as 11 later
beer_pivot_all = beer_pivot_all.loc[beer_pivot_all["Brand_Rank_by_Country"] < 11]
beer_pivot_all = beer_pivot_all[["Country_and_Brand", "Brand_Rank_by_Country"]]

#Create unique column by merge country & Brand standarised
df_["Country_and_Brand"] = df_["Country"] + "_" + df_["Brand_standarization"]

## Work on the master datafram, vlookup - by merge function left join:
df_rank = pd.merge(df_, beer_pivot_all, on = "Country_and_Brand", how = "left") 

## if NaN values in the Brand_Rank_by_Country_y column, replace the values as 11th rank i.e. 11
df_rank["Brand_Rank_by_Country_y"] = df_rank["Brand_Rank_by_Country_y"].fillna(11)
df_rank["Brand_Rank_by_Country_y"] = df_rank["Brand_Rank_by_Country_y"].astype(int) # Convert rank into int from float

df_rank.drop(['Brand_Rank_by_Country_x', 'Country_and_Brand'], axis = 1, inplace = True)
df_rank = df_rank.rename(columns = {'Brand_Rank_by_Country_y': 'Brand_Rank_by_Country'})


In [36]:
df_final = df_rank[['Country', 'Data_Source', 'Advertiser_parent_company', 'Brand',
       'Category', 'Media_type', 'Year', 'Month', 'Spend_Local_Currency',
       'Exchange_Rate', 'Spend_US_Currency', 'Category_Standardized',
       'Advertiser_parent_standarized', 'Brand_standarization',
       'Parent_competitor', 'Top_Brand_Competitors_and_ABI',
       'Focused_Brand_Competitors', 'Top10_Brands_by_Market', 'Media_Summary',
       'Media_type_standarized', 'Month_number', 'Month_standarization',
       'Focused_Global_Brand', 'Global_Parent', 'ABI_Parent_SOS',
       'Heineken_Brand_SOS', 'Amstel_Brand_SOS', 'Digital_Traditional',
       'Heineken_Parent_SOS', 'Brand_Rank_by_Country', 'Market_Group']]

In [37]:
### Dividing data into two fiels based on Groups:
df_final_1 = df_final[df_final["Market_Group"] == "Group I"]
df_final_2 = df_final[df_final["Market_Group"] == "Group II"]

In [41]:
# Initial Dataframe, Final dataframe shape
print(df_initial.shape)
print(df_final.shape)
print(df_final_1.shape)
print(df_final_2.shape)

(507209, 11)
(507209, 31)
(72943, 31)
(434266, 31)


**** DO NOT EXECUTE CODE AFTER THIS LINE***

### CODE TESTING

In [42]:
os.chdir(r"C:\ABI\2023 Competitive\2023 Competitive through Q4\Ad Spend\Processed")
print(os.getcwd())
#df_final.to_excel("Q2-Competitive Ad Spend-V4.xlsx", index=False)

#df.to_excel("2023 FY_test.xlsx", index=False) # test

df_final_1.to_excel("2023 FY Adspend Processed Group_I.xlsx", index=False)
df_final_2.to_excel("2023 FY Adspend Processed Group_II.xlsx", index=False)

# End the timer
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Print the elapsed time
print(f"Elapsed time: {elapsed_time} seconds")
print(f"Elapsed time: {elapsed_time/60} minutes")

C:\ABI\2023 Competitive\2023 Competitive through Q4\Ad Spend\Processed
Elapsed time: 660.5141861438751 seconds
Elapsed time: 11.008569769064586 minutes
